In [1]:
import os
%pwd  # this tell us which path we are currently working , so based on the below output path we are working under the research file
#os.getcwd()  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../")
os.chdir("C:\datascience End to End Projects\End-to-End-Heart-Disease-Application-main")
%pwd

'C:\\datascience End to End Projects\\End-to-End-Heart-Disease-Application-main'

In [2]:
# This is called the entity 
from dataclasses import dataclass # here i imported the dataclass from the dataclasses
from pathlib import Path  # here i imported path from pathlib

@dataclass(frozen=True)
class FeatureSelectionConfig:
    root_dir: Path  # these are variables which are present inside the config.yaml file data_transformation code part and here iam mentioning inside the entity of the class
    train_data_path: Path
    test_data_path: Path
    


In [3]:
from PROJECTML.constants import *
from PROJECTML.utils.common import read_yaml, create_directories

In [4]:
# this is same part of the code in every step 
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    # only this part get changes in every step, only defining the get_data_transformation_config get changes according to which step we are performing like 01_data_ingestion,02_data_validation
    def get_feature_selection_config(self) -> FeatureSelectionConfig:
        config = self.config.feature_selection

        create_directories([config.root_dir])

        feature_selection_config = FeatureSelectionConfig(
            root_dir=config.root_dir,  # here iam returning these 2 varaibles by using this code 
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
        )

        return feature_selection_config

In [5]:
import os
from PROJECTML import logger
from sklearn.model_selection import train_test_split


In [6]:
# here i defined the component of DataTransformationConfig below
import pandas as pd


class FeatureSelection:
    def __init__(self, config: FeatureSelectionConfig):
        self.config = config

     

    def feature_selection(self):
        train_csv=pd.read_csv(self.config.train_data_path)
        test_csv=pd.read_csv(self.config.test_data_path)
        self.train_csv_features = train_csv[train_csv.columns.drop(['RestingBP','RestingECG'])]
        self.test_csv_features= test_csv[test_csv.columns.drop(['RestingBP','RestingECG'])]


                
# here i have defined the tarin_test_split below for performing the train_test_split
    def train_test_spliting(self):

        self.train_csv_features.to_csv(os.path.join(self.config.root_dir, "Modified_train.csv"),index = False) # here it saves the train and test data in csv format inisde the artifacts-> transformation folder
        self.test_csv_features.to_csv(os.path.join(self.config.root_dir, "Modified_test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(self.train_csv_features.shape) # this logs the information about that how many training and testing samples i have 
        logger.info(self.test_csv_features.shape)

        print(self.train_csv_features.shape)
        print(self.test_csv_features.shape)


In [7]:
try:
    config = ConfigurationManager() # here iam initlizing my ConfigurationManager
    feature_selection_config = config.get_feature_selection_config() # and here iam getting my get_data_transformation_config()
    feature_selection = FeatureSelection(config=feature_selection_config) # here iam passing my data_transformation_config it means iam calling this data_transformation_config
    feature_selection.feature_selection()
    feature_selection.train_test_spliting() # here performing the train_test_split()
except Exception as e: # this part of code will raise error if anything goes wrong
    raise e

[2024-02-22 01:59:59,776: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-22 01:59:59,780: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 01:59:59,784: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-22 01:59:59,787: INFO: common: created directory at: artifacts]
[2024-02-22 01:59:59,790: INFO: common: created directory at: artifacts/feature_selection]


[2024-02-22 01:59:59,852: INFO: 2832826836: Splited data into training and test sets]
[2024-02-22 01:59:59,854: INFO: 2832826836: (734, 10)]
[2024-02-22 01:59:59,855: INFO: 2832826836: (184, 10)]
(734, 10)
(184, 10)
